# Overview

pyvespa provides a python API to Vespa. The library’s primary goal is to allow for faster prototyping and facilitate Machine Learning experiments for Vespa applications: 

1. Build and deploy a Vespa application using pyvespa API.
2. Connect to an existing Vespa application and run queries from python.
3. Import a Vespa application package from files and use pyvespa to access it.

## Build and deploy

The [getting-started](https://pyvespa.readthedocs.io/en/latest/getting-started-pyvespa.html) notebook
is a good primer on how to create an application, feed data and run queries.

##  Query a running Vespa application

When a Vespa application is already running, one can instantiate the [Vespa](https://pyvespa.readthedocs.io/en/latest/reference-api.html#vespa.application.Vespa) class with the endpoint.

Refer to [connect-to-vespa-instance](https://pyvespa.readthedocs.io/en/latest/connect-to-vespa-instance.html)
to connect to and application and run queries.

## Deploy from Vespa config files

The `pyvespa` API provides a subset of the functionality available in Vespa. The reason is that `pyvespa` is meant to be used as an experimentation tool for Information Retrieval (IR) and not for building production-ready applications. So, the python API expands based on the needs to replicate common use cases that often require IR experimentation.

If the application requires functionality or fine-tuning not available in `pyvespa`, simply build it directly using Vespa configuration files as shown in [many examples](https://docs.vespa.ai/en/getting-started.html) on Vespa docs. But even in this case, one can still get value out of `pyvespa` by deploying it from python based on the Vespa configuration files stored on disk.

Clone and deploy the news search app covered in this [Vespa tutorial](https://docs.vespa.ai/en/tutorials/news-3-searching.html):

In [10]:
!git clone https://github.com/vespa-engine/sample-apps.git $WORK_DIR/sample-apps

Cloning into '/Users/tmartins/projects/vespa/pyvespa//sample-apps'...
remote: Enumerating objects: 17599, done.
remote: Counting objects: 100% (376/376), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 17599 (delta 180), reused 225 (delta 92), pack-reused 17223
Receiving objects: 100% (17599/17599), 87.09 MiB | 1.64 MiB/s, done.
Resolving deltas: 100% (10100/10100), done.
Checking out files: 100% (544/544), done.


The Vespa configuration files of the news search app are stored in `sample-apps/news/app-3-searching/`:

In [11]:
!tree $WORK_DIR/sample-apps/news/app-3-searching/

/Users/tmartins/projects/vespa/pyvespa//sample-apps/news/app-3-searching/
├── hosts.xml
├── schemas
│   └── news.sd
└── services.xml

1 directory, 3 files


Deploy to a Docker container from disk:

In [12]:
import os
from vespa.deployment import VespaDocker

disk_folder_news = os.path.join(os.getenv("WORK_DIR"), "sample-apps/news/app-3-searching/")
vespa_docker_news = VespaDocker(
    disk_folder=disk_folder_news, 
    port=8090
)
app = vespa_docker_news.deploy_from_disk(application_name="news")

Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for configuration server.
Waiting for application status.
Waiting for application status.
Finished deployment.


Again, `app` holds an instance of the Vespa class just like the first example. This can be used to feed and query the application just deployed. 

Clean up:

In [13]:
from shutil import rmtree

rmtree(disk_folder_news, ignore_errors=True)
vespa_docker_news.container.stop()
vespa_docker_news.container.remove()

## Summary

These are three different ways to connect to a `Vespa` application from python using the `pyvespa` library. Those methods provide great workflow flexibility, allowing users to quickly get started with pyvespa experimentation. This also enables modification of Vespa config files to include features not available in the pyvespa API, without losing the ability to experiment with the added features.